In [7]:
from pathlib import Path
from fastai.vision.all import *
from yogapose.users.utils import get_pose_name
from yogapose.models import User, Post

In [8]:
from yogapose import create_app
app = create_app()
app.app_context().push()

/home/gautam/YogaPose/venv/lib/python3.7/site-packages/flask_sqlalchemy/__init__.py:834: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [9]:
username='Gautam'
# Get posts of user
user_query = User.query.filter_by(username=username)
user_posts_query = Post.query.filter_by(author=user_query.first_or_404())
user_posts_list = user_posts_query.all()

In [16]:
def yogatodo(user_posts_list: list) -> tuple:
    """ Return tuple of 
    1) list of completed asanas and score and 
    2) list of remaining asanas and score=0

    Parameters:
    user_posts_list (list): Posts of a user
    """
    # Get average score for users posts
    data = [(user_post.pose_name, user_post.pose_score) for user_post in user_posts_list]
    df_avg = pd.DataFrame.from_records(data, columns=['pose_name', 'score']).groupby('pose_name').max('score')
    df_avg.reset_index(inplace=True)
    df_avg = df_avg[df_avg.pose_name != 'no pose']

    # Get list of poses from model
    path = Path('/home/gautam/YogaPose/yogapose').joinpath('pkls')
    learn_inf = load_learner( path.joinpath('y82-resnet18-multi.pkl') )
    poses = learn_inf.dls.vocab

    #Get poses not done
    user_poses_set = set(df_avg.pose_name)
    not_done_poses = {get_pose_name(pose) for pose in poses}.difference(user_poses_set)
    not_done_list = [(not_done_pose, 0) for not_done_pose in not_done_poses]

    return (df_avg.values.tolist(), not_done_list)

In [17]:
done_avg, not_done = yogatodo(user_posts_list=user_posts_list)

In [18]:
done_avg

[['extended revolved triangle pose', 100],
 ['sitting pose 1 (normal)', 100],
 ['tree pose', 100],
 ['virasana', 57]]